In [52]:
import os
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep

df = pd.DataFrame()
TT, T, T0, T1, T2, T3, T4 = list(), list(), list(), list(), list(), list(), list()
url = "https://sc.scourt.go.kr/sc/krsc/criterion/criterion_01/murder_01.jsp"
raw = requests.get(url, headers={'User-Agent' : 'Mozilla/5.0'})
html = BeautifulSoup(raw.text, 'html.parser')
titles = html.select("div#container ul:nth-of-type(1) li:nth-of-type(3) ul li")

for t in titles:
    
    url_t =  'https://sc.scourt.go.kr/' + t.select_one('a').attrs['href']
    raw_t = requests.get(url_t, headers={'User-Agent' : 'Mozilla/5.0'})
    html_t = BeautifulSoup(raw_t.text, 'html.parser')
    
    title = t.select_one('a').text

    if '·' in title:
        if html_t.select("div#tabs-0 ol li ul.cirList01"):
            continue
        sub = html_t.select("div#tabs-0 ol li")
        tables = html_t.select("div#tabs-0 div table")
        for i in range(len(sub)):
            cont = tables[2*i].select('tr td')
            try:
                tt, t, t0, t1, t2, t3, t4= title.replace('범죄 양형기준', ''), sub[i].text, cont[0].text, cont[1].text, cont[2].text, cont[3].text, cont[4].text
            except:
                tt, t, t0, t1, t2, t3, t4= title.replace('범죄 양형기준', ''), sub[i].text, '0', cont[0].text, cont[1].text, cont[2].text, cont[3].text
            TT.append(tt);T.append(t);T0.append(t0);T1.append(t1);T2.append(t2);T3.append(t3);T4.append(t4)
        continue
   
    table = html_t.select_one("div.tableType01")
    tbody = table.select('table tbody tr')
    for tb in tbody:
        cont = tb.select('td')
        try:
            tt, t0, t1, t2, t3, t4= title.replace('범죄 양형기준', ''), cont[0].text, cont[1].text, cont[2].text, cont[3].text, cont[4].text
        except:
            tt, t0, t1, t2, t3, t4= title.replace('범죄 양형기준', ''), '0', cont[0].text, cont[1].text, cont[2].text, cont[3].text
        TT.append(tt);T.append(str());T0.append(t0);T1.append(t1);T2.append(t2);T3.append(t3);T4.append(t4)
        
df['title'] = TT;df['sub'] = T;df['c0'] = T0;df['c1'] = T1;df['c2'] = T2;df['c3'] = T3;df['c4'] = T4

df_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'condition_table'}.xlsx")
df.to_excel(df_name)